In [ ]:
import pysam 
import numpy as np
from scipy import stats
import sys

import os
import pandas as pd
import numpy as np
import argparse
import glob

import tensorflow as tf
from sklearn.preprocessing import StandardScaler

from dagmm import DAGMM

from scipy.special import logsumexp
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline

## set variable

In [3]:
test_bam = '/zfssz6/ST_MCHRI/BIGDATA/P18Z10200N0124/NIPT_CNV/final_file/NA12878/SRR098401_1.sorted.rmdup.realign.BQSR.bam'
ref_fasta = '/zfssz6/ST_MCHRI/BIGDATA/database/BGI-seq500_OSS_download/human_reference/hg38/Homo_sapiens_assembly38.fasta'

# varaible set
bin_dir = '/zfssz6/ST_MCHRI/BIGDATA/P18Z10200N0124/NIPT_CNV/NIPT_CNV/GMM_1cnn_MODEL_train/bin'
base_dir = '/zfssz6/ST_MCHRI/BIGDATA/P18Z10200N0124/NIPT_CNV/NIPT_CNV/GMM_1cnn_MODEL_train/run/winsize_1000k'
cp_ratio = base_dir + '/test_sample/SRR098401_1.sorted.rmdup.realign.BQSR.bam.NA12878_winsize_1000000.correct_ratio.txt'
model_dir = base_dir + '/model_dir'
truth_cnv = bin_dir + '/ALL.wgs.mergedSV.v8.20130502.svs.genotypes.GRCh38.vcf.cnvs.dat'

chrom_ref = ['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10',
'chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chr21','chr22','chrX','chrY','chrM']

colors={0:'r', 1:'g', 2: 'b', 3:'k',4:'m',5:'c',6:'y'}
legends={0:'DUP', 1: 'NEU', 2: 'DEL', 3:'DUP/DEL'}
bin_size=[10000,100000,1000000]
win_size=[10,15,20,30,50]
win = bin_size[2]
chrs = ['chr'+str(i) for i in range(1,23)] 

## define function

In [4]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)


In [5]:
def gen_copy(cp_ratio, npysave):

    ws = win_size[2]
    cy_ratio_mat = []
    cp_df = pd.read_csv(cp_ratio, sep='\t')

    for chrn in chrs:
        chr_df = cp_df[(cp_df['space']==chrn) & (cp_df['copy'])].sort_values(by=['start'])
        i_copy = chr_df['copy'].values
        # add 0 at the beginning of sequence
        tmp_copy = np.append([0]*(ws-1),i_copy)
        i_cy_matrix= rolling_window(tmp_copy,ws)
        if len(cy_ratio_mat) == 0:
            cy_ratio_mat = i_cy_matrix
        else:
            cy_ratio_mat = np.concatenate((cy_ratio_mat,i_cy_matrix),axis=0)

    np.save(os.path.join(npysave, cp_ratio.split("/")[-1] + '_bins20.npy'), cy_ratio_mat)

In [6]:

def get_model(model_dir):
    
    model_dagmm = DAGMM(
        comp_hiddens=[32,16,1], comp_activation=tf.nn.tanh, # 16
        est_hiddens=[8,16,3], est_activation=tf.nn.tanh, est_dropout_ratio=0.25,
        epoch_size=100, minibatch_size=1024,normalize=False #1000
    )
    model_dagmm.restore(model_dir)
    return model_dagmm

In [7]:
def get_raw_copy(cp_ratio):

    ws = win_size[2]
    test_df = pd.read_csv(cp_ratio, sep='\t')
    
    pred_chrms={}
    for chrn in chrs:

        chr_df = test_df[(test_df['space']==chrn) & (test_df['copy'])].sort_values(by=['start'])
        i_copy = chr_df['copy'].values
        start_pos=chr_df['start'].values
        end_pos=chr_df['end'].values
        reads = chr_df['reads'].values
        gcs = chr_df['gc'].values
        mapps = chr_df['map'].values
    #     print(chrn)

        d={'start':start_pos,'end':end_pos,'copy':i_copy,'gc':gcs,'reads':reads,'maps':mapps}
        pred_chrms[chrn] = pd.DataFrame(data=d)
    return pred_chrms

In [8]:
def get_pred(cp_ratio, model_dagmm):
    
    ws = win_size[2]
    test_df = pd.read_csv(cp_ratio, sep='\t')
     
    pred_chrms={}
    for chrn in chrs:

        chr_df = test_df[(test_df['space']==chrn) & (test_df['copy'])].sort_values(by=['start'])
        i_copy = chr_df['copy'].values
        start_pos=chr_df['start'].values
        end_pos=chr_df['end'].values
        reads = chr_df['reads'].values
        gcs = chr_df['gc'].values
        mapps = chr_df['map'].values
        # add 0 at the beginning of sequence
        tmp_copy = np.append([0]*(ws-1),i_copy)
        i_cy_matrix= rolling_window(tmp_copy,ws)   


        energy,logit_probs = model_dagmm.predict(i_cy_matrix)
        lse = logsumexp(logit_probs,axis=0)
        log_resp=logit_probs-lse[np.newaxis,:]
        comp_prob = np.exp(log_resp)
        i_comp=np.argmax(comp_prob,axis=0)


        d={'start':start_pos,'end':end_pos,'pred':i_comp,'copy':i_copy,'gc':gcs,'reads':reads,'maps':mapps}
        pred_chrms[chrn] = pd.DataFrame(data=d)
        
    return pred_chrms

In [9]:
# 'HG01136'
def get_turth_cnv(truth_cnv, sampleId):
    
    cnvs_annos = pd.read_csv(truth_cnv,
               sep='\t',usecols=['CHROM', 'POS', 'ID', 'ALT', 'SVTYPE',
           'SVLEN', 'END','AF', 'SUPP_N_HET','SUPP_N_HOM_ALT','SUPP_HETS','SUPP_HOM_ALTS'])

    test_cnvs = cnvs_annos[cnvs_annos['SUPP_HETS'].str.contains(sampleId) 
                           | cnvs_annos['SUPP_HOM_ALTS'].str.contains(sampleId)]

    test_cnvs.sort_values(by=['CHROM','POS'],inplace=True)
    test_cnvs.reset_index(drop=True, inplace=True)


    adjstart = np.floor(test_cnvs['POS'].values*1.0/win) * win + 1
    adjstart = adjstart.astype('int')
    test_cnvs['ADJPOS']= adjstart
    adjend = np.ceil(test_cnvs['END'].values*1.0/win) * win
    adjend = adjend.astype('int')
    test_cnvs['ADJEND'] = adjend
    test_cnvs['NWIN']=(test_cnvs['ADJEND']-test_cnvs['ADJPOS']+1)//win 
    test_cnvs['ADJTYPE']=test_cnvs['SVTYPE']
    test_cnvs.loc[(test_cnvs['SVTYPE']=='CNV') & ( ~test_cnvs['ALT'].str.contains('CN0')),'ADJTYPE']='DUP'
    re_str = '('+sampleId+',[0-9]\|[0-9])'
    test_cnvs['TARGET_HET_GT']=test_cnvs['SUPP_HETS'].str.extract(re_str)
    test_cnvs['TARGET_HOM_GT']=test_cnvs['SUPP_HOM_ALTS'].str.extract(re_str)
    test_cnvs = test_cnvs[['CHROM','POS','ID','ALT','ADJTYPE','END','SVLEN','AF','ADJPOS','ADJEND','NWIN','TARGET_HET_GT','TARGET_HOM_GT']]
    
    return test_cnvs

In [10]:
def get_normal_cnv(test_cnvs,pred_chrms,chromosom):
    
    tru_chr1_cnvs = test_cnvs[test_cnvs['CHROM']==chromosom]
    pred_chr1_cnv=pred_chrms['chr' + str(chromosom)]
    pred_chr1_cnv['truth']='NEU'
    pred_chr1_cnv['ID']=''
    pred_chr1_cnv['svlen']=0
    pred_chr1_cnv['POS']=0
    pred_chr1_cnv['END']=0
    pred_chr1_cnv['TARGET_HET_GT']=''
    pred_chr1_cnv['TARGET_HOM_GT']=''


    for i,x in tru_chr1_cnvs.iterrows():

        pred_chr1_cnv.loc[(pred_chr1_cnv['start']>=x['ADJPOS']) & (pred_chr1_cnv['end']<=x['ADJEND']),'truth'] = x['ADJTYPE']
        pred_chr1_cnv.loc[(pred_chr1_cnv['start']>=x['ADJPOS']) & (pred_chr1_cnv['end']<=x['ADJEND']),'ID'] = x['ID']
        pred_chr1_cnv.loc[(pred_chr1_cnv['start']>=x['ADJPOS']) & (pred_chr1_cnv['end']<=x['ADJEND']),'svlen'] = int(x['SVLEN'])
        pred_chr1_cnv.loc[(pred_chr1_cnv['start']>=x['ADJPOS']) & (pred_chr1_cnv['end']<=x['ADJEND']),'AF'] = x['AF']
        pred_chr1_cnv.loc[(pred_chr1_cnv['start']>=x['ADJPOS']) & (pred_chr1_cnv['end']<=x['ADJEND']),'POS'] = x['POS']
        pred_chr1_cnv.loc[(pred_chr1_cnv['start']>=x['ADJPOS']) & (pred_chr1_cnv['end']<=x['ADJEND']),'END'] = int(x['END'])
        pred_chr1_cnv.loc[(pred_chr1_cnv['start']>=x['ADJPOS']) & (pred_chr1_cnv['end']<=x['ADJEND']),'TARGET_HET_GT'] = x['TARGET_HET_GT']
        pred_chr1_cnv.loc[(pred_chr1_cnv['start']>=x['ADJPOS']) & (pred_chr1_cnv['end']<=x['ADJEND']),'TARGET_HOM_GT'] = x['TARGET_HOM_GT']
    
    return pred_chr1_cnv[pred_chr1_cnv['truth']=='NEU']

In [11]:

def seq2num(seqs):
    
    tar_list = {'':0, 'N':0, 'A':1,'T':2,'C':3,'G':4}
    results = []
    
    for base in seqs:
        result = int(tar_list[base.upper()]) 
        results.append(result)
    return results

In [12]:
def check_GC(seqs):
    
    tar_list = {'':0, 'G':1, 'C':1,'A':-1,'T':-1,'N':0}
    results = []
    
    for base in seqs:
        result = int(tar_list[base.upper()]) 
        results.append(result)
    return results

In [13]:
# calculate feature
def gen_feature(chr, start, end, samfile, reffile):

    bin_size = end - start

    #variable initial
    feature_list = ['feature_1','feature_2', 'feature_3', 'feature_3', 'feature_4', 'feature_5', 'feature_10', 'feature_11']
    feature_1 = np.zeros(bin_size, int)
    feature_2 = np.zeros(bin_size, int)
    feature_3 = np.zeros(bin_size, int)
    feature_4= np.zeros(bin_size, int)
    feature_5= np.zeros(bin_size, int)

    feature_10= np.zeros(bin_size, int)
    feature_11= np.zeros(bin_size, int)

    positions = np.array(range(start,end))

    #check if no reads maped within regions
    bin_readcounts = samfile.count(chr, start, end)
    if bin_readcounts > 1:

        #get ref message
        ref_bin = reffile.fetch(chr, start, end)
        base_ref_sequences = seq2num(list(ref_bin))
        base_ref_is_gc = check_GC(list(ref_bin))                                

        feature_5= np.array(base_ref_is_gc)
        feature_10= np.array(base_ref_sequences)

        #get sample message

        iter = samfile.pileup(chr, start, end, truncate = True)
        index = 0

        for pileupcolumn in iter:

            pos = pileupcolumn.pos
            base_num_aligned = np.mean(pileupcolumn.get_num_aligned())
            
            if base_num_aligned == 0:
                
                #position without reads coverage
                base_seq_qualities = 0
                base_mapping_qualities = 0
                base_query_sequences = 0
                base_is_gc = 0
                
            else:
                
#                 print('query_sequences: ', pileupcolumn.get_query_sequences())
#                 print(len(pileupcolumn.get_query_sequences()))
#                 print('ok')

                base_query_sequences = 0
                base_is_gc = stats.mode(check_GC(pileupcolumn.get_query_sequences()))[0][0]


                base_query_sequences = stats.mode(seq2num(pileupcolumn.get_query_sequences()))[0][0]
                base_is_gc = stats.mode(check_GC(pileupcolumn.get_query_sequences()))[0][0]
                    
#                 print('query_sequences', pileupcolumn.get_query_sequences())
#                 print('mapping_qualities: ',pileupcolumn.get_mapping_qualities())
#                 print('query_qualities()',pileupcolumn.get_query_qualities())
                base_seq_qualities =  np.mean(pileupcolumn.get_query_qualities())
                base_mapping_qualities = np.mean(pileupcolumn.get_mapping_qualities())
                
                
                

            positions[index] = pos
            feature_1[index] = base_num_aligned
            feature_2[index] = base_seq_qualities
            feature_3[index] = base_mapping_qualities
            feature_4[index] = base_is_gc
            feature_11[index] = base_query_sequences
            index = index + 1

        feature = dict(zip(feature_list, [sys._getframe().f_back.f_locals[a] for a in feature_list]))
    else:

        feature = 'Inf'

    return feature


## run test

In [14]:
test_cnvs = get_turth_cnv(truth_cnv, 'NA12878')

/zfssz5/BC_PS/liuyu6/software/anaconda3/envs/ai/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/zfssz5/BC_PS/liuyu6/software/anaconda3/envs/ai/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/zfssz5/BC_PS/liuyu6/software/anaconda3/envs/ai/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

In [15]:
# use_bin.head(10)
chromosom = 1

test_cnvs['AF'] = test_cnvs['AF'].str.split(',',expand=True)[0]
AF_test_cnvs = test_cnvs[test_cnvs['AF'].astype(np.float) >= 0.5]
use_bin = AF_test_cnvs[['CHROM', 'POS', 'END']]
use_cnv_bin = use_bin[use_bin['CHROM']==chromosom]

# print(len(use_bin))
pred_chrms = get_raw_copy(cp_ratio)
use_normal = get_normal_cnv(test_cnvs,pred_chrms,chromosom)
use_normal_bin = use_normal[['start', 'end']]
use_cnv_bin.head(10)

,CHROM,POS,END
4,1,11622816,11623132.0
7,1,21459925,21460202.0
12,1,35161503,35161830.0
13,1,36267681,36267914.0
20,1,58278238,58279154.0
22,1,67542287,67543128.0
23,1,72300660,72346132.0
24,1,72312854,72318911.0
28,1,85514778,85540461.0
37,1,109644537,109648770.0


In [ ]:
print(len(use_cnv_bin))
print(len(use_normal_bin))

In [17]:
#read file
samfile =  pysam.AlignmentFile(test_bam, 'rb')
reffile =  pysam.FastaFile(ref_fasta)

In [18]:
# generage cnv region features

cnv_feature = {}
cnv_reature_len = []
for index, row in use_cnv_bin.iterrows():

    contig = 'chr' + str(row['CHROM'])
    start = row['POS']
    end = int(row['END'])
    
    bin_position = contig + ':' + str(start) + '-' + str(end)

    feature = gen_feature(contig, start, end, samfile, reffile)
    cnv_reature_len.append(len(feature))
    cnv_feature[bin_position] = feature


In [ ]:
# generage normal region features
normal_feature= {}
normal_feature_len = []
for index, row in use_normal_bin.iterrows():

    contig = 'chr' + str(chromosom)
    start = row['start']
    end = int(row['end'])
    
    bin_position = contig + ':' + str(start) + '-' + str(end)

    feature = gen_feature(contig, start, end, samfile, reffile)
    normal_feature_len.append(len(feature))
    normal_feature[bin_position] = feature

In [ ]:
print(len(cnv_reature_len))
print(len(normal_feature_len))